<a href="https://colab.research.google.com/github/201820722/SNS_HEAT/blob/main/Sns_trend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data'

/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data


In [19]:
!pip install apyori

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=26c70ee900edbc6c155a37a64d84e853bd6ad12243643a969e3b6025f55fbf81
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [171]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import os
import json
from apyori import apriori
import numpy as np

# 데이터 읽기 및 통합
def read_and_combine_json_files(file_paths):
    extracted_data = []

    for json_file in file_paths:
        with open(json_file, 'r') as file:
            data = json.load(file)

        extracted_item1 = []
        extracted_item2 = []
        extracted_item1.append(data['header']['dialogueInfo']['single_topic'])
        extracted_item1.append(data['header']['participantsInfo'][0]['gender'])
        extracted_item1.append(data['header']['participantsInfo'][0]['age'])
        extracted_item2.append(data['header']['dialogueInfo']['single_topic'])
        extracted_item2.append(data['header']['participantsInfo'][1]['gender'])
        extracted_item2.append(data['header']['participantsInfo'][1]['age'])
        extracted_data.append(extracted_item1)
        extracted_data.append(extracted_item2)

    df = pd.DataFrame(extracted_data, columns=['topic', 'sex', 'age'])
    # df = df[df['age'] == '20대'] ## 20대만 분석
    return df

# Association Rule 생성
def generate_association_rules(data):
    data = data.drop(['sex'], axis=1)
    data = list(data)
    te = TransactionEncoder()
    te_ary = te.fit(data).transform(data)
    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)


    return rules

# 기계학습 모델 구성 (TF-IDF)
def build_machine_learning_model(df):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['topic'])

    y = df['age']  # 대화 주제를 나이대로 설정

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = MultinomialNB()
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions))

    return model

# 입력에 대한 대화 트렌드 분석
def analyze_conversation_trends(input_gender, input_age, rules):
    # 입력된 성별과 나이에 따른 대화 주제 트렌드 예측
    trend_prediction = rules[(rules['antecedents'].apply(lambda x: input_gender in x)) &
                             (rules['antecedents'].apply(lambda x: input_age in x))]

    return trend_prediction['consequents']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [165]:
# 여러 개의 JSON 파일 경로 설정
json_path = '/content/drive/MyDrive/Colab Notebooks/Data_Mining/TP/Data'
json_file_paths = [pos_json for pos_json in os.listdir(json_path) if pos_json.endswith('.json')]

# 주어진 데이터 읽기
df = read_and_combine_json_files(json_file_paths)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic,sex,age
0,주거와 생활,여성,20대
1,주거와 생활,남성,20대
2,주거와 생활,남성,20대
3,주거와 생활,여성,20대
7,미용과 건강,여성,20대
...,...,...,...
5991,미용과 건강,여성,20대
5994,일과 직업,여성,20대
5995,일과 직업,여성,20대
5996,식음료,여성,20대


In [166]:
# df = df.drop(['age'], axis = 1)
# df1 = df[df['sex'] == '여성']
# df2 = df[df['sex'] == '남성']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [167]:
# df1.reset_index(drop=True, inplace=True)
# df1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic,sex
0,주거와 생활,여성
1,주거와 생활,여성
2,미용과 건강,여성
3,주거와 생활,여성
4,주거와 생활,여성
...,...,...
2457,미용과 건강,여성
2458,일과 직업,여성
2459,일과 직업,여성
2460,식음료,여성


In [168]:
# df2.reset_index(drop=True, inplace=True)
# df2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,topic,sex
0,주거와 생활,남성
1,주거와 생활,남성
2,주거와 생활,남성
3,식음료,남성
4,개인 및 관계,남성
...,...,...
620,개인 및 관계,남성
621,개인 및 관계,남성
622,일과 직업,남성
623,식음료,남성


In [172]:
# Association Rule 생성
conversation_df = generate_association_rules(df)
print(conversation_df)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: ignored

In [ ]:
# 기계학습 모델 생성 및 평가
ml_model = build_machine_learning_model(conversation_df)

In [ ]:
# 입력에 대한 대화 트렌드 분석 예시
input_gender = '여성'  # 예시 성별
input_age = '20대'    # 예시 나이
trend_prediction = analyze_conversation_trends(input_gender, input_age, association_rules_df)
print(f"Trend Prediction for {input_gender}, {input_age}:\n", trend_prediction)